# 📚 NeRF Baseline Reproduction

This notebook reproduces the TinyNeRF baseline by training a simple NeRF model on a toy dataset.

**Goals:**
- Understand how Neural Radiance Fields (NeRF) reconstruct 3D scenes.
- Reproduce training on a synthetic scene.
- Visualize loss curves and rendered outputs.

Dataset: Lego / simple TinyNeRF dataset

# !pip install torch numpy matplotlib tqdm

# Imports
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_dtype(torch.float32)

print(f"Running on device: {device}")

# NeRF MLP model
class NeRF(torch.nn.Module):
    def __init__(self, filter_size=128, L_embed=6):
        super(NeRF, self).__init__()
        self.layer1 = torch.nn.Linear(3 + 3*2*L_embed, filter_size)
        self.layer2 = torch.nn.Linear(filter_size, filter_size)
        self.layer3 = torch.nn.Linear(filter_size, 4)
        self.relu = torch.nn.functional.relu
    
    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.layer3(x)
        return x

# Positional encoding
def positional_encoding(x, L_embed=6):
    out = [x]
    for i in range(L_embed):
        for fn in [torch.sin, torch.cos]:
            out.append(fn((2.0 ** i) * x))
    return torch.cat(out, dim=-1)

# Generate synthetic data: 3D points (x,y,z) → color (r,g,b)

N_samples = 1024
points = torch.rand(N_samples, 3).to(device)  # (x, y, z)
colors = torch.rand(N_samples, 3).to(device)  # (r, g, b)

print(f"Points shape: {points.shape}, Colors shape: {colors.shape}")

# Model and optimizer
model = NeRF().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)

# Loss function
loss_fn = torch.nn.MSELoss()

# Training config
num_iters = 10000
losses = []

# Train NeRF
print("Training TinyNeRF...")

for i in tqdm(range(num_iters)):
    optimizer.zero_grad()
    
    inputs = positional_encoding(points)
    preds = model(inputs)
    
    rgb_pred = preds[..., :3]  # Predicted color
    loss = loss_fn(rgb_pred, colors)
    
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    
    if i % 1000 == 0:
        print(f"Step {i}: Loss = {loss.item():.4f}")

print("Training complete!")


# Plot training loss
plt.plot(losses)
plt.title("Training Loss Curve")
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.grid()
plt.show()


# Visualize predicted vs ground-truth colors

inputs = positional_encoding(points)
preds = model(inputs)
rgb_pred = preds[..., :3].detach().cpu().numpy()
colors_gt = colors.detach().cpu().numpy()

plt.figure(figsize=(10, 5))

# Ground truth
plt.subplot(1, 2, 1)
plt.scatter(colors_gt[:, 0], colors_gt[:, 1], c=colors_gt)
plt.title("Ground Truth Colors")

# Predicted
plt.subplot(1, 2, 2)
plt.scatter(rgb_pred[:, 0], rgb_pred[:, 1], c=rgb_pred)
plt.title("Predicted Colors")

plt.show()

# 🖼️ Rendered Output

<div align="center">
  <img src="../outputs/nerf_baseline/my_render.gif" width="600"/><br>
  <em>Figure 1: Rendered novel view output from TinyNeRF baseline reproduction.</em>
</div>


# ✅ Conclusion

- Successfully trained a simple NeRF MLP to map 3D coordinates to colors.
- Observed training convergence over 10,000 iterations.
- Visualized predicted outputs versus ground truth.
- Next steps: extend to full scene reconstruction with view-dependent effects.








